# Chi-Squared Calculation Comparison

This notebook compares the Chi-Squared values returned by SNCosmo against Chi-Squared values calculated by the analysis pipeline.

In [ ]:
%matplotlib inline

import sys
from warnings import warn

import numpy as np
import sncosmo
from SNData.csp import dr3
from SNData.des import sn3yr
from astropy.table import Table, join
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from analysis_pipeline import get_fit_results, get_priors, SN91bgSource

for module in (dr3, sn3yr):
    module.download_module_data()
    module.register_filters(force=True)


## Manually Calculating Chi-Squared

In addition to comparing the SNCosmo and pipeline chi-squared values, we also define a `calc_chisq` function so that we can experiment with alternative chi-squared equations. For clarity, we will call this the "manual" chi-squared.

In [ ]:
def calc_chisq(data, model):
    """
    Calculate the chi-squared for a given data table and model
    
    Args:
        data  (Table): An SNCosmo input table
        model (Model): An SNCosmo Model
        
    Returns:
        The un-normalized chi-squared
        The number of data points used in the calculation
    """

    while True:
        try:
            # Model flux and keep only non-zero values
            data['model_flux'] = [model.bandflux(b, t) for b, t in zip(data['band'], data['time'])]
            data = data[data['model_flux'] > 0]
            chisq = np.sum(((data['model_flux'] - data['flux']) / data['fluxerr']) ** 2)
            return chisq, len(data)

        except ValueError as err:
            # Remove bands that are out of model range
            data = data[data['band'] != err.args[0].split()[1][1:-1]]


## Fitting with All Data

We begin by considering chi-squared values calculated from fits using all available band data. We arbitrarily pick a target from the CSP DR3 data release and fit it's light curve with the Salt2.4 model.

In [ ]:
test_id = '2007S'
model = sncosmo.Model('salt2')
data = dr3.get_data_for_id(test_id, format_sncosmo=True)
priors = get_priors(dr3, model).loc[test_id]

model.set(z=data.meta['redshift'])
bounds = dict()
for p in model.param_names:
    bounds[p] = (priors[f'{p}_min'], priors[f'{p}_max'])
    model.update({p: priors[p]})

fit_result, fit_model = sncosmo.fit_lc(data, model, ['t0', 'x0', 'x1', 'c'], bounds=bounds, phase_range=(-20,50))
sncosmo.plot_lc(data, fit_model)


Next we compare the various chi-squared values.

In [ ]:
sncosmo_chisq = fit_result['chisq'] / fit_result['ndof']
print('Chi-Squared from fit_lc:', sncosmo_chisq)

man_chisq, num_points = calc_chisq(data, fit_model)
man_dof = fit_result['ndof'] - len(data) + num_points
print('Manual Chi-squared using all data:', man_chisq / man_dof)

pipeline_data = get_fit_results(dr3, model, 4)[0].loc[test_id]
print('Pipeline Chi-squared using all data:', 
      pipeline_data['chi'] / pipeline_data['dof'])


Self-implemented lc-fitting plots for clarity.

In [ ]:
allbands = data.copy()

fig = plt.figure(figsize=(10,12))
ax = [plt.subplot(3, 2, i + 1) for i in range(6)]

b = ['csp_dr3_u', 'csp_dr3_B', 'csp_dr3_g', 'csp_dr3_V', 'csp_dr3_r', 'csp_dr3_i']
plot_color = ['darkblue', 'blue', 'aqua', 'lightblue', 'lightgreen', 'yellow']
for i in range(6):
    data = allbands.copy()
    data = data[data['band'] == b[i]]
    ax[i].errorbar(data['time'], data['flux'], yerr=data['fluxerr'], fmt='.', markersize=3, capsize=2)
    xtime = np.arange(data['time'].min(), data['time'].max(),0.1)
    yflux = [fit_model.bandflux(data['band'][0], t) for t in xtime]
    ax[i].plot(xtime, yflux, c=plot_color[i], label=b[i])
    ax[i].set_xlabel('Time')
    ax[i].set_ylabel('Flux')
    ax[i].legend()

## Fitting Individual Bands

In addition to fitting all available bands at once, we also fit individual band passes and look at the chi-squared values summed over the individual bands.

In [ ]:
band_sncosmo_chisq = dict()
band_manual_chisq = dict()
for band in set(data['band']):
    band_data = data[data['band'] == band]
    try:
        fit_result, fitted_model = sncosmo.fit_lc(band_data, model, ['t0', 'x0', 'x1', 'c'], phase_range=(-15,50))
        
    except RuntimeError:
        continue
    
    
    # Calculate chi-squared
    man_chisq, num_points = calc_chisq(band_data, fitted_model)
    man_dof = fit_result['ndof'] - len(band_data) + num_points
    band_manual_chisq[band] = man_chisq / man_dof
    band_sncosmo_chisq[band] = fit_result['chisq'] / fit_result['ndof']

print('SNCosmo Chi-Squared:')
print(band_sncosmo_chisq)

print('\nSNCosmo Chi-Squared summed over bands:')
print(sum(i for i in band_sncosmo_chisq.values()))

print('\nManual Chi-Squared:')
print(band_manual_chisq)

print('\nManual Chi-Squared summed over bands:')
print(sum(i for i in band_manual_chisq.values()))
